In [33]:
# Common Imports
import numpy as np
import pandas as pd
from datetime import datetime, timedelta, date
import os

# Data Import Module
%run ./Module_DataImport.ipynb

## Deployment Information
In the code cell are variables that can be tuned based on the deployment, they are described in more detail below:
- student_list: a integer list of the students included in this study. Valid student numbers are between 2 and 9, inclusive.
- start_date: the first date that you would like to include in the summary. The earliest date that can be chosen that will ensure at least one student has data is 03/11/2019.
- end_date: the last date to include in the summary. No more data are available after 04/15/2019.

The start and end dates are defaulted to the earliest and latest dates, respectively, given in the description above. 

In [18]:
student_list = np.arange(2,10,1)
start_date = '03/11/2019'
end_date = '04/15/2019'

## Data Import

### Indoor Environmental Quality Data
In the code cell below, the indoor environmental quality data is imported from the ```beacon_data``` directory available for Students 2 through 9. The variables include:
1. Temperature
2. Relative Humidity$^1$
3. PM2.5

The functions for importing the data are stored in the ```Module_DataImport.pynb``` notebook in the same directory as this master notebook. 

$^1$Temperature and Relative Humidity are imported together and stored in the same dataframe. Collectively, they are referred to as "thermal comfort"

In [19]:
TC_byStudent = pd.Series()
Concentration_byStudent = pd.Series()
for n in student_list:
    print('Importing IEQ Data for: Student',n)
    TC_byStudent['Student' + str(n)] = ieqImport(str(n),'sht31d',['time','rh','temp'])
    Concentration_byStudent['Student' + str(n)] = ieqImport(str(n),'pms5003',['time','PM2.5'],[0,2])

Importing IEQ Data for: Student 2
Number of datapoints: 205
Number of datapoints: 1
Importing IEQ Data for: Student 3
Number of datapoints: 7832
Number of datapoints: 102
Importing IEQ Data for: Student 4
Number of datapoints: 9780
Number of datapoints: 111
Importing IEQ Data for: Student 5
Number of datapoints: 6512
Number of datapoints: 13
Importing IEQ Data for: Student 6
Number of datapoints: 9231
Number of datapoints: 7249
Importing IEQ Data for: Student 7
Number of datapoints: 9629
Number of datapoints: 7564
Importing IEQ Data for: Student 8
Number of datapoints: 7390
Number of datapoints: 3749
Importing IEQ Data for: Student 9
Number of datapoints: 9775
Number of datapoints: 7679


### Sleep Quality Data
There are two types of sleep quality data, gathered from different sources:
1. Sleep Stages from Fitbit
2. Perceived Sleep Quality from Beiwe EMA Surveys

Both datasets are imported below in the code cell.

In [36]:
# Data Import Module
%run ./Module_DataImport.ipynb

SS_byStudent = pd.Series()
PSQ_byStudent = pd.Series()
for n in student_list:
    print('Importing Sleep Quality Data for: Student',n)
    SS_byStudent['Student' + str(n)] = fitbitImport(str(n),'S' + str(n) + '_SleepStages.csv',['ID','LogID','Time','Level','ShortWakes','Stage_Label'])
    

Importing Sleep Quality Data for: Student 2
Data/Student2/S2_SleepStages.csv
Reading in raw data
Empty DataFrame
Columns: [ID, LogID, Time, Level, ShortWakes, Stage_Label]
Index: []
Number of datapoints: 0
Importing Sleep Quality Data for: Student 3
Data/Student3/S3_SleepStages.csv
Reading in raw data
Empty DataFrame
Columns: [ID, LogID, Time, Level, ShortWakes, Stage_Label]
Index: []
Number of datapoints: 0
Importing Sleep Quality Data for: Student 4
Data/Student4/S4_SleepStages.csv
Reading in raw data
Empty DataFrame
Columns: [ID, LogID, Time, Level, ShortWakes, Stage_Label]
Index: []
Number of datapoints: 0
Importing Sleep Quality Data for: Student 5
Data/Student5/S5_SleepStages.csv
Reading in raw data
Empty DataFrame
Columns: [ID, LogID, Time, Level, ShortWakes, Stage_Label]
Index: []
Number of datapoints: 0
Importing Sleep Quality Data for: Student 6
Data/Student6/S6_SleepStages.csv
Reading in raw data
Empty DataFrame
Columns: [ID, LogID, Time, Level, ShortWakes, Stage_Label]
Inde